In [8]:
import numpy as np
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data

In [9]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [70]:
def xavier_init(shape, uniform=True):
    n_inputs = shape[0]
    n_outputs = shape[1]
    if uniform:
        init_range = np.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform(shape, minval = -init_range, maxval = init_range, dtype = tf.float32)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal(shape, stddev=stddev)

class SimpleAutoencoder(object):
    def __init__(self, n_input, n_hidden, transfer_function = tf.nn.relu,
                optimizer = tf.train.AdamOptimizer(learning_rate=0.001), scale = 0.01):
        
        self.n_input = n_input
        self.n_hidden = n_hidden
        
        self.transfer = transfer_function
        
        self.scale = tf.constant(scale)
        
        self.weights = self._initialize_weights()
        
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        
        noise_data = tf.add(self.x, scale * tf.random_normal((n_input,)))
        
        self.hidden = self.transfer(tf.add(tf.matmul(noise_data, self.weights['w1']), self.weights['b1']))
        
        #self.decoder = tf.matmul(self.hidden, self.weights['w2']) + self.weights['b2']
        self.decoder = tf.nn.softmax(tf.matmul(self.hidden, self.weights['w2']) + self.weights['b2'])
        
        self.cost = tf.reduce_sum(tf.pow(tf.subtract(self.decoder, self.x), 2.0))
        #self.cost = tf.reduce_mean(-tf.reduce_sum(self.x * tf.log(self.decoder), reduction_indices = [1]))
    
        #self.optimizer = optimizer.minimize(self.cost)
        self.optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(self.cost)
            
        self.y = tf.placeholder(tf.float32, [None, 10])
        
        self.pred_y = tf.nn.softmax(tf.matmul(self.hidden, self.weights['w3']) + self.weights['b3'])

        #self.hidden_drop = tf.nn.dropout(self.hidden, 0.75)
    
        #self.pred_y = tf.nn.softmax(tf.matmul(self.hidden_drop, self.weights['w3']) + self.weights['b3'])
  
        self.pred_cost = tf.reduce_mean(-tf.reduce_sum(self.y * tf.log(self.pred_y), reduction_indices = [1]))
    
        #self.pred_optimizer = optimizer.minimize(self.pred_cost)
        self.pred_optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(self.pred_cost)
        
        self.correct_pred = tf.equal(tf.argmax(self.pred_y, 1), tf.argmax(self.y, 1))

        self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))
        
        self.sess = tf.Session()
        
        self.sess.run(tf.global_variables_initializer())    
    
    def _initialize_weights(self):
        weights = dict()
        weights['w1'] = tf.Variable(xavier_init([self.n_input, self.n_hidden], uniform=True))
        weights['b1'] = tf.Variable(tf.zeros([self.n_hidden]))
        weights['w2'] = tf.Variable(xavier_init([self.n_hidden, self.n_input], uniform=True))
        weights['b2'] = tf.Variable(tf.zeros([self.n_input]))
        weights['w3'] = tf.Variable(xavier_init([self.n_hidden, 10], uniform=True))
        weights['b3'] = tf.Variable(tf.zeros([10]))
        return weights
    
    def train(self, X):
        cost, _ = self.sess.run((self.cost, self.optimizer),
                                feed_dict = {self.x: X})
        return cost
    
    def pred_train(self, X, Y):
        cost, _ = self.sess.run((self.pred_cost, self.pred_optimizer),
                                feed_dict = {self.x: X, self.y: Y})
        return cost
    def pred_test(self, X, Y):
        return self.sess.run(self.accuracy, feed_dict = {self.x:mnist.test.images, self.y:mnist.test.labels})
    
    
    def getWeights(self, s):
        return self.sess.run(self.weights[s])

autoencoder = SimpleAutoencoder(n_input = 784, n_hidden = 300)

In [71]:
training_epochs = 10000
batch_size = 128
for epoch in range(training_epochs):
    
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    pred_cost = autoencoder.pred_train(batch_xs, batch_ys)
    acc = autoencoder.pred_test(mnist.test.images, mnist.test.labels)
    cost = autoencoder.train(batch_xs)
    
    if epoch % 200 == 0:
        print(pred_cost, acc, cost)
        # w3 = autoencoder.getWeights('w3')
        # print(np.min(w3), np.max(w3))

2.4048114 0.1263 11102.981
0.19968891 0.9224 11007.644
0.13991922 0.9496 10896.789
0.1655643 0.9565 11123.721
0.09168452 0.9611 11221.191
0.037404977 0.9705 11300.115
0.18293929 0.971 11188.677
0.11147476 0.9699 11524.828
0.03599199 0.9748 10782.232
0.06462801 0.975 11170.167
0.097896986 0.9773 11242.126
0.15032366 0.9758 10994.654
0.046942323 0.9773 10979.986
0.06344323 0.9768 11265.444
0.047065042 0.9776 10671.857
0.0668446 0.9786 10322.025
0.036787607 0.9786 11217.576
0.03748689 0.9792 11009.731
0.011198699 0.9808 11225.528
0.023867665 0.981 11504.851
0.018647894 0.9811 10882.911
0.015583331 0.9808 11061.827
0.023653002 0.9782 10412.644
0.0055297688 0.9795 11107.547
0.010931587 0.9824 10764.207
0.07769946 0.9799 11050.549
0.001992417 0.9808 11225.545
0.0060652383 0.9803 10760.182
0.0031475055 0.9818 11721.6
0.010667109 0.9819 10949.738
0.003665465 0.9807 10795.297
0.0082237795 0.9826 10585.24
0.009027703 0.9809 10993.989
0.0033787515 0.9823 11034.369
0.0060623838 0.982 10861.296
0.0